In [379]:
import pandas as pd
import numpy as np

df=pd.read_csv('data1.csv')
df.columns=['ID','Name','Email','Phone_no','Country', 'Age','Salary','Join_Date']

In [380]:
# Cleaning Name section
df['Name']=(df['Name']
            .str.title()
            .str.strip(' ,.')
            .replace([np.nan,'','unknown'],np.nan))



In [381]:
# cleaning email
pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
df.loc[18,'Email']='hashir'

df.loc[~df['Email'].str.fullmatch(pattern,na=False),'Email']='Invalid Mail'


In [382]:
# cleaning phone no

df['Phone_no']=(df['Phone_no']
                .str.replace(r'(x|ext\.?).*$', '', regex=True, case=False)
                .str.replace(r'[^0-9+\-]', '', regex=True)
                .str.replace(r'-{2,}', '-', regex=True)
                .str.strip('-')
                .apply(lambda x: '+' + x.lstrip('+0') if pd.notna(x) else x))


# done fully using gpt//?

# droping it because it is not handling perfectly

df.drop(columns=['Phone_no'],inplace=True)
                



In [383]:
# cleaning salary and age

df["Salary"]=df['Salary'].replace(np.nan,df['Salary'].mean()).astype('int')
df['Age']=df['Age'].fillna(df['Age'].interpolate())
df['Age']=df['Age'].replace(np.nan,df['Age'].mean()).astype(int)


In [384]:
# cleaning time

# df['Join_Date']=df['Join_Date'].str.replace(r'([+-]\d{2}:?\d{2}|Z)$','',regex=True)
df['Join_Date'] = (df['Join_Date']
                   .astype(str)
                   .str.replace(r'([+-]\d{2}:?\d{2}|Z)$', ' ', regex=True)
                    .str.replace(r'[\/\.]', '-', regex=True)
                    .str.strip()
                    .str.replace(r'[^\w\s:-]', '', regex=True))
df['Join_Date'] = pd.to_datetime(df['Join_Date'], errors='coerce', infer_datetime_format=True)
df['Join_Date'] = df['Join_Date'].fillna(pd.Timestamp('2024-01-01'))


df


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3504\3863387375.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Join_Date'] = pd.to_datetime(df['Join_Date'], errors='coerce', infer_datetime_format=True)


,ID,Name,Email,Country,Age,Salary,Join_Date
0,681,Heather Davila,igill@bryant-roberts.com,Inda,50,54349,2019-04-01
1,1103,Margaret Campbell,bergjack@martinez-reeves.com,India,50,66604,2024-01-01
2,395,Veronica Wilson,jeffreychavez@gmail.com,United States,80,66604,2021-07-15
3,931,Matthew Mcguire,angelahorne@cordova.com,India,79,186523,2024-01-01
4,498,Anne Gentry,april19@dawson.com,Pak,78,156760,2021-07-15
...,...,...,...,...,...,...,...
1245,1045,William Carr,scottmendez@gomez.com,Pakistan,74,91550,2021-07-15
1246,1096,Nicole Donovan,nmoore@hotmail.com,Germny,30,57504,2018-02-05
1247,1131,Deborah Brooks,Invalid Mail,Pak,30,44270,2017-03-31
1248,861,Hannah Davis,candace91@yahoo.com,Canada,30,162748,2017-07-19


In [385]:
import pandas as pd
import pycountry

#
manual_map = {
    'pk': 'Pakistan',
    'pak': 'Pakistan',
    'usa': 'United States',
    'u.s.a': 'United States',
    'Uas':'United States',
    'uas': 'United States',
    'america': 'United States',
    'uk': 'United Kingdom',
    'u.k': 'United Kingdom',
    'england': 'United Kingdom',
    'uae': 'United Arab Emirates',
    'emirates': 'United Arab Emirates',
    'russia': 'Russian Federation',
    'south korea': 'Korea, Republic of',
    'north korea': "Korea, Democratic People's Republic of",
    'viet nam': 'Vietnam',
    'brasil': 'Brazil',
    'syria': 'Syrian Arab Republic',
    'palestine': 'State of Palestine',
    'germny':'Germany',
    'inda':'India'

}

# 🧠 Function to clean country names
def standardize_country(name):
    if not isinstance(name, str):
        return name
                
    name_clean = name.strip().lower()
    
    # Check manual map first
    if name_clean in manual_map:
        return manual_map[name_clean]
    
    # Match using pycountry codes and names
    for country in pycountry.countries:
        if name_clean in [
            country.name.lower(),
            country.alpha_2.lower(),
            country.alpha_3.lower()
        ]:
            return country.name
    
    # Return formatted name if no match found
    return name.title()

# 🧹 Apply cleaning to the 'country' column
df['Country']=df['Country'].apply(standardize_country)




In [ ]:
df.sort_values(by='ID',ascending=True,inplace=True)

TypeError: unsupported operand type(s) for -: 'int' and 'str'

In [387]:
final=df.to_csv('cleaned_data.csv',index=False)